In [1]:
from accelerate import Accelerator, FullyShardedDataParallelPlugin
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import pandas as pd
import json
from datasets import Dataset, DatasetDict
import evaluate


/root/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_json_from_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
        return pd.DataFrame(data) 

data = read_json_from_file("./data/refined_hindi_data.json")
dataset = Dataset.from_pandas(data)

dataset_dict = dataset.train_test_split(test_size=0.2, seed=42)

train_dataset, test_dataset = dataset_dict['train'], dataset_dict['test']
train_dataset


Dataset({
    features: ['instruction', 'output'],
    num_rows: 11948
})

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")

def tokenize_function(examples):
    inputs = tokenizer(examples["instruction"], examples["output"], truncation=True, max_length=512, padding="max_length", return_tensors="pt")
    inputs["labels"] = inputs["input_ids"].clone()
    return inputs

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["instruction", "output"])
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=["instruction", "output"])
tokenized_train_dataset



Map: 100%|██████████| 2988/2988 [00:01<00:00, 2243.34 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 11948
})

In [4]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=2, 
    evaluation_strategy="no",
    do_train=False,
    fp16=True, 
    gradient_checkpointing=True

)

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B", device_map="auto")
trainer = Trainer(
    model=model, 
    args=training_args, 
    eval_dataset=tokenized_test_dataset, 
    compute_metrics=compute_metrics
)


eval_results = trainer.evaluate()





/root/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


OutOfMemoryError: CUDA out of memory. Tried to allocate 9.85 GiB. GPU 0 has a total capacity of 23.68 GiB of which 8.84 GiB is free. Process 3725736 has 14.83 GiB memory in use. Of the allocated memory 11.04 GiB is allocated by PyTorch, and 3.55 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch
torch.cuda.device_count()